## Problems 1-6

### Import Packages

In [141]:
import numpy as np
import scipy as sp
from scipy import linalg

### Optimization Class

In [142]:
class optimization:
    """Accepts arrays c, A, and b to solve a linear optimization
    problem in standard form"""
    def __init__(self, c, A, b):
        zero1 = np.zeros_like(c)
        if np.any(np.dot(A,zero1) > b):
            raise ValueError("Problem not feasible at the origin")
        else:
            # Initialize the input variables
            self.c = c
            self.A = A
            self.b = b
            # Create initial L list
            m,n = np.shape(A)
            L1 = np.arange(m-1)
            L2 = np.arange(n,m+n)
            self.L = np.concatenate((L2,L1))
        
    def tableau(self):
        c_star = (-self.c).T
        zero2 = np.zeros_like(self.b.T)
        zero3 = np.array([0])
        one1 = np.array([1])
        row1 = np.concatenate((zero3, c_star, zero2, one1))
        one2 = np.eye(np.shape(A)[0])
        b2 = np.reshape(b,(np.shape(b)[0],1))
        zero4 = np.zeros_like(b2)
        row2 = np.concatenate((b2, A, one2, zero4),axis=1)
        row1_star = np.reshape(row1, (1, np.shape(row1)[0]))
        self.T = np.concatenate((row1_star, row2), axis=0)
        return self.T
    
    def bland(self, T):
        """takes as an input the tableau matrix T from the tableau function
        uses Bland rule to determine the row and column to pivot
        returns pivot row and pivot column"""
        i = 0
        a = T[0,i]
        while a >= 0:
            i += 1
            a = T[0,i]
        self.col = i
        if np.any(T[:,i] > 0):
            ratios = np.zeros(np.shape(self.A)[0]+1)
            for i in range(np.shape(ratios)[0]):
                if T[i,0]/T[i,self.col] > 0:
                    ratios[i] = T[i,0]/T[i,self.col]
                else:
                    ratios[i] = np.inf
        else:
            raise ValueError("No non-negative values for Bland method")
        self.row = np.argmin(ratios)
        index = T[self.row, self.col]
        return self.row, self.col
    
    def pivot(self, T):
        index_A = self.row-1
        index_B = np.where(self.L==self.col-1)
        self.L[index_A], self.L[index_B] = self.L[index_B], self.L[index_A]
        center = T[self.row, self.col]
        T[self.row,:] = (1/center)*T[self.row,:]
        v = T[self.row,:]
        m,n = np.shape(T)
        for i in range(m):
            if i != self.row:
                alpha = T[i, self.col]
                T[i,:] = T[i,:]-alpha*v
        return T, self.L
    
    def solve(self):
        self.T = self.tableau()
        i = 0
        while np.any(self.T[0,1:] < 0) or i >= 100:
            self.bland(self.T)
            self.pivot(self.T)
            i += 1
        solution = self.T[0,0]
        basic = dict()
        nonbasic = dict()
        m,n = np.shape(self.T)
        length = len(self.L)
        basic = dict()
        for i in range(m-1):
            basic[self.L[i]] = self.T[i+1,0]
        nonbasic = dict()
        for j in range(m-1,length):
            nonbasic[self.L[j]] = 0
        print("Converged after " + str(i) + " iterations")
        print("Maximum Value = " + str(solution))
        return (solution, basic, nonbasic)        

### Implementation

In [143]:
A = np.array([[1,-1],[3,1],[4,3]])
b = np.array([2,5,7])
c = np.array([3,2])

In [144]:
problem = optimization(c,A,b)
problem.solve()

Converged after 2 iterations
Maximum Value = 5.2


(5.2000000000000002,
 {0: 1.5999999999999996, 1: 0.20000000000000034, 2: 0.60000000000000053},
 {3: 0, 4: 0})

## Problem 7

In [155]:
data = np.load('productMix.npz')
A = data['A']
p = data['p']
M = data['m']
d = data['d']
m,n = A.shape
for i in range(n):
    A = np.concatenate((A, np.eye(1, j, i)))
b = np.concatenate((M,d))
problem = optimization(p, A, b)
solution = problem.solve()
print(solution)


Converged after 6 iterations
Maximum Value = 7453.59649123
(7453.5964912280688, {0: 10.0, 1: 6.192982456140351, 2: 12.0, 3: 1.7894736842105277, 6: 0.96596491228070169, 8: 13.807017543859651, 10: 8.2105263157894726}, {9: 0, 4: 0, 5: 0, 7: 0})


/Users/Geoff/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:46: RuntimeWarning: divide by zero encountered in double_scalars
/Users/Geoff/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:47: RuntimeWarning: divide by zero encountered in double_scalars
